# Fine-Tuning Neural Network hyperparameters

In [13]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [21]:
housing = fetch_california_housing()
x = housing.data 
y = housing.target
x_train_ful, x_test, y_train_ful, y_test = train_test_split(x, y , test_size = 0.2, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_ful, y_train_ful, test_size = 0.2, random_state = 42)
x_new = x_train[:3]

In [36]:
x_new.shape
y_train[:3]

array([1.769, 1.733, 2.047])

In [32]:
#create simple sequential model for univariate regression(only one output neuron)

def build_model(n_hidden = 1, n_neurons = 30, learning_rate = 3e-3, input_shape = [8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation = 'relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr= learning_rate)
    model.compile(loss = 'mse', opimizer = optimizer)
    return model
    
    

In [33]:
#create keras regressor based on build_model
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)
keras_reg

In [34]:
keras_reg.fit(x_train, y_train, epochs= 20, validation_data = (x_valid, y_valid),
              callbacks = [keras.callbacks.EarlyStopping(patience = 10)])
mse_test = keras_reg.score(x_test, y_test)
y_pred = keras_reg.predict(x_new)

Train on 13209 samples, validate on 3303 samples
Epoch 1/20
13209/13209 [==============================] - 3s 209us/sample - loss: 1625.5292 - val_loss: 2.7788
Epoch 2/20
13209/13209 [==============================] - 3s 211us/sample - loss: 31.1622 - val_loss: 1.6888
Epoch 3/20
13209/13209 [==============================] - 2s 174us/sample - loss: 29.7797 - val_loss: 4.2164
Epoch 4/20
13209/13209 [==============================] - 3s 216us/sample - loss: 29.5068 - val_loss: 2.0696
Epoch 5/20
13209/13209 [==============================] - 3s 221us/sample - loss: 29.8706 - val_loss: 5.3709
Epoch 6/20
13209/13209 [==============================] - 3s 211us/sample - loss: 26.5299 - val_loss: 5.2677
Epoch 7/20
13209/13209 [==============================] - 3s 213us/sample - loss: 27.7497 - val_loss: 2.2601
Epoch 8/20
13209/13209 [==============================] - 3s 221us/sample - loss: 24.9016 - val_loss: 320.9577
Epoch 9/20
13209/13209 [==============================] - 2s 184us/sample -

In [35]:
y_pred

array([1.917608 , 1.9758469, 1.5120668], dtype=float32)

# finding best hyperparameters

In [45]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [46]:
param = {
    'n_neurons': np.arange(1,100),
    'n_hidden':[0,1,2,3],
    'learning_rate':reciprocal(3e-4, 3e-2)
    
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param, n_iter=3, cv =2)
rnd_search_cv.fit(x_train, y_train, epochs = 20, validation_data = (x_valid, y_valid),
              callbacks = [keras.callbacks.EarlyStopping(patience = 10)])

Train on 6604 samples, validate on 3303 samples
Epoch 1/20
6604/6604 [==============================] - 2s 253us/sample - loss: 149854.4995 - val_loss: 38845.5184
Epoch 2/20
6604/6604 [==============================] - 1s 196us/sample - loss: 7373.7061 - val_loss: 93.0348
Epoch 3/20
6604/6604 [==============================] - 1s 190us/sample - loss: 42.3447 - val_loss: 24.6886
Epoch 4/20
6604/6604 [==============================] - 1s 206us/sample - loss: 16.9676 - val_loss: 11.2349
Epoch 5/20
6604/6604 [==============================] - 2s 235us/sample - loss: 7.1955 - val_loss: 6.9748
Epoch 6/20
6604/6604 [==============================] - 2s 236us/sample - loss: 4.7694 - val_loss: 5.0247
Epoch 7/20
6604/6604 [==============================] - 1s 199us/sample - loss: 3.6217 - val_loss: 3.6717
Epoch 8/20
6604/6604 [==============================] - 1s 173us/sample - loss: 3.5051 - val_loss: 3.1967
Epoch 9/20
6604/6604 [==============================] - 1s 147us/sample - loss: 3.1876 

Train on 6605 samples, validate on 3303 samples
Epoch 1/20
6605/6605 [==============================] - 2s 238us/sample - loss: 1322106.0386 - val_loss: 976104.6003
Epoch 2/20
6605/6605 [==============================] - 1s 186us/sample - loss: 622960.3794 - val_loss: 381867.2805
Epoch 3/20
6605/6605 [==============================] - 1s 204us/sample - loss: 186590.6374 - val_loss: 72600.7368
Epoch 4/20
6605/6605 [==============================] - 1s 216us/sample - loss: 25587.7020 - val_loss: 8202.9904
Epoch 5/20
6605/6605 [==============================] - 1s 202us/sample - loss: 5560.0426 - val_loss: 4013.5750
Epoch 6/20
6605/6605 [==============================] - 2s 233us/sample - loss: 2508.2887 - val_loss: 1508.2598
Epoch 7/20
6605/6605 [==============================] - 2s 231us/sample - loss: 725.4058 - val_loss: 243.0363
Epoch 8/20
6605/6605 [==============================] - 2s 231us/sample - loss: 56.1160 - val_loss: 27.7416
Epoch 9/20
6605/6605 [===========================

Train on 6604 samples, validate on 3303 samples
Epoch 1/20
6604/6604 [==============================] - 2s 342us/sample - loss: 202965.9343 - val_loss: 70419.1585
Epoch 2/20
6604/6604 [==============================] - 1s 173us/sample - loss: 17296.9542 - val_loss: 171.2480
Epoch 3/20
6604/6604 [==============================] - 1s 168us/sample - loss: 4.9372 - val_loss: 148.5212
Epoch 4/20
6604/6604 [==============================] - 1s 194us/sample - loss: 3.6840 - val_loss: 123.8731
Epoch 5/20
6604/6604 [==============================] - 1s 157us/sample - loss: 2.9679 - val_loss: 104.0332
Epoch 6/20
6604/6604 [==============================] - 1s 204us/sample - loss: 2.6321 - val_loss: 91.8176
Epoch 7/20
6604/6604 [==============================] - 1s 202us/sample - loss: 2.3568 - val_loss: 80.2405
Epoch 8/20
6604/6604 [==============================] - 1s 203us/sample - loss: 2.2474 - val_loss: 55.5492
Epoch 9/20
6604/6604 [==============================] - 1s 199us/sample - loss: 

Train on 6605 samples, validate on 3303 samples
Epoch 1/20
6605/6605 [==============================] - 2s 251us/sample - loss: 859156.9390 - val_loss: 559766.2651
Epoch 2/20
6605/6605 [==============================] - 1s 124us/sample - loss: 308562.5340 - val_loss: 137858.7127
Epoch 3/20
6605/6605 [==============================] - 2s 228us/sample - loss: 45752.4223 - val_loss: 1165.5667
Epoch 4/20
6605/6605 [==============================] - 1s 207us/sample - loss: 110.5652 - val_loss: 308.1699
Epoch 5/20
6605/6605 [==============================] - 2s 237us/sample - loss: 63.7158 - val_loss: 288.4394
Epoch 6/20
6605/6605 [==============================] - 2s 244us/sample - loss: 61.6588 - val_loss: 283.0380
Epoch 7/20
6605/6605 [==============================] - 1s 200us/sample - loss: 58.6742 - val_loss: 277.1643
Epoch 8/20
6605/6605 [==============================] - 1s 186us/sample - loss: 56.9361 - val_loss: 268.6712
Epoch 9/20
6605/6605 [==============================] - 1s 16

Train on 6604 samples, validate on 3303 samples
Epoch 1/20
6604/6604 [==============================] - 2s 321us/sample - loss: 32695.6085 - val_loss: 130.5148
Epoch 2/20
6604/6604 [==============================] - 1s 171us/sample - loss: 58.8131 - val_loss: 87.4541
Epoch 3/20
6604/6604 [==============================] - 1s 166us/sample - loss: 48.3321 - val_loss: 90.8344
Epoch 4/20
6604/6604 [==============================] - 1s 161us/sample - loss: 59.4609 - val_loss: 62.4026
Epoch 5/20
6604/6604 [==============================] - 1s 163us/sample - loss: 69.4152 - val_loss: 54.6874
Epoch 6/20
6604/6604 [==============================] - 1s 165us/sample - loss: 49.1799 - val_loss: 266.0254
Epoch 7/20
6604/6604 [==============================] - 1s 161us/sample - loss: 58.5149 - val_loss: 36.0649
Epoch 8/20
6604/6604 [==============================] - 1s 166us/sample - loss: 49.6779 - val_loss: 30.8963
Epoch 9/20
6604/6604 [==============================] - 1s 164us/sample - loss: 47.

Train on 6605 samples, validate on 3303 samples
Epoch 1/20
6605/6605 [==============================] - 2s 366us/sample - loss: 503.4564 - val_loss: 11.9222
Epoch 2/20
6605/6605 [==============================] - 1s 166us/sample - loss: 55.2742 - val_loss: 54.8933
Epoch 3/20
6605/6605 [==============================] - 1s 174us/sample - loss: 53.2541 - val_loss: 3.3817
Epoch 4/20
6605/6605 [==============================] - 1s 161us/sample - loss: 44.3096 - val_loss: 7.8095
Epoch 5/20
6605/6605 [==============================] - 1s 164us/sample - loss: 48.2295 - val_loss: 15.5442
Epoch 6/20
6605/6605 [==============================] - 1s 150us/sample - loss: 44.3420 - val_loss: 6.1074
Epoch 7/20
6605/6605 [==============================] - 1s 151us/sample - loss: 46.8554 - val_loss: 2.2923
Epoch 8/20
6605/6605 [==============================] - 1s 149us/sample - loss: 48.4335 - val_loss: 2.9495
Epoch 9/20
6605/6605 [==============================] - 1s 151us/sample - loss: 47.5986 - va

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x00000263C24A4CC8>, as the constructor either does not set or modifies parameter learning_rate

In [47]:
rnd_search_cv.best_params_

{'learning_rate': 0.0018584821819282672, 'n_hidden': 0, 'n_neurons': 97}

In [49]:
rnd_search_cv.best_score_

-1.4990026054008125